In [43]:
import geopandas as gpd
import pandas as pd 
from sklearn import preprocessing 
import networkx as nx
import numpy as np
import math
import random
import community
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from modularity_maximization import partition
from modularity_maximization.utils import get_modularity
from modutils import mod_calc
assert torch.cuda.is_available()
device = torch.device('cuda')
gpus = [0]
from typing import Dict
from scipy.sparse import coo_matrix

In [7]:
import dgl
import dgl.function as fn
from dgl import DGLGraph

Using backend: pytorch


In [86]:
from tqdm import tqdm

In [60]:
K = random.randint(5, 10)
M = random.randint(1, 100)
average_cluster_size = random.randint(20, 40)
center_x_list = []
center_y_list = []
for i in range(K):
    center_x_list.append(round(10*random.random(), 4))
    center_y_list.append(round(10*random.random(), 4))
node_groundtruth = {}
node_location = {}
idx = 0
for i in range(K):
    for j in range(average_cluster_size):
        node_groundtruth[idx] = i
        node_location[idx] = [center_x_list[i] + round(random.gauss(0, 1), 4), center_y_list[i] + round(random.gauss(0, 1), 4)]
        idx = idx + 1
N = len(node_groundtruth)
G = nx.Graph()
G.add_nodes_from(list(range(N)))
loc_np = np.array(list(node_location.values()))
x_np = loc_np[:, 0].reshape((1, -1))
y_np = loc_np[:, 1].reshape((1, -1))
distance_np = np.square(x_np - x_np.T)+np.square(y_np - y_np.T)
k_nearest = {}
k = int(average_cluster_size/2)
for i in range(N):
    k_near_pre = list(np.argsort(distance_np[i])[:k+1:])
    k_near_pre.remove(i)
    k_nearest[i] = k_near_pre
for i in range(N):
    for j in k_nearest[i]:
        G.add_edge(i, j, weight = np.exp(-1/2*distance_np[i, j]))
G.to_undirected()
omega = 1/K*np.ones(K)
beta = 1/M*np.ones(M)
FF = omega.reshape((1, -1))*beta.reshape((-1, 1))
FF = FF*K*M
X = np.zeros((N, M))
for i in range(M):
    for j in range(K):
        tmp = np.random.dirichlet(np.ones(average_cluster_size), size=1)*FF[i, j]
        X[j*average_cluster_size:(j+1)*average_cluster_size:, i] = tmp

In [5]:
class CDRAE(nn.Module):
    def __init__(self, A_hat, num_feat, num_hidden, W_prior, weight_list):
        super(CDRAE, self).__init__()
        f = nn.Softmax(dim=2)
        self.graph_num = A_hat.shape[0]
        self.num_feat = num_feat # 特征数 f
        self.num_hidden = num_hidden # 隐含数 h
        self.weight_list = weight_list
        self.A_hat = A_hat
        self.d = self.A_hat.sum(axis=1).float().contiguous().view(self.graph_num, -1, 1)
        self.dT = self.A_hat.sum(axis=2).float().contiguous().view(self.graph_num, -1, 1)
        self.ddT = torch.mul(self.d.expand(self.graph_num, self.num_feat, self.num_feat), self.dT.expand(self.graph_num, self.num_feat, self.num_feat))
        self.B = (self.A_hat - torch.div(self.ddT, torch.sum(torch.sum(self.A_hat, axis=-1), axis=-1).view(-1, 1, 1))).float()
        self.B_norm = torch.div(self.B, torch.sum(torch.sum(self.A_hat, axis=-1), axis=-1).view(-1, 1, 1))
        self.W_0 = nn.Parameter(f(torch.ones(self.graph_num, self.num_feat, self.num_hidden).to(device)+W_prior)) # [1][f*h]

    def forward(self, A_hat,temp, epoch): # X貌似暂时没有用到
        global featureSelector # 
        global weight_feature # 
        
        featureSelector = self.W_0 # [1][f*h]
        weight_feature_multi = torch.zeros(self.W_0.size()).to(device)
        x = 1*(epoch+1) # x次取平均,目测这个目的是为了稳定。
        for j in range(self.graph_num):
            results = torch.zeros((self.num_feat, self.num_hidden)).to(device) # [0][f*h]
            for i in range(x):
                results += F.gumbel_softmax(self.W_0[j],tau=temp,hard=True)
            weight_feature_multi[j] = results/x # [-][f*h]
        weight_feature = torch.mean(weight_feature_multi, axis=0)
        
        H = torch.mm(torch.mm(weight_feature.T, torch.mean(self.A_hat*self.weight_list.view(-1,1,1), axis=0)), weight_feature) # weight_feature 就是我们要求的U
        H = torch.div(H, H.sum(axis=0)) # 按照每一列进行归一化，列为axis=0的方向。
        m = nn.Softmax(dim=0)
        return m(H)
    
def lossFn(output): 
    return torch.trace(-torch.log(output))

def th_kl_div(p, q):
    return torch.mean(torch.sum((0.00001+p)*torch.log(torch.div(0.00001+p, 0.00001+q)), axis=1))*p.shape[1]

In [61]:
clusters_number=K
feature=X
multi_graph=[G]
alpha_1=1
alpha_2=0.5
alpha_3=0.01
weight_list=[1]
alpha=2
epoch_num=150

In [62]:
gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')
DG = dgl.DGLGraph()
DG.from_networkx(G)

In [78]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)
        
class GCN_model(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN_model, self).__init__()
        self.layer = GCNLayer(in_feats, out_feats)
        # self.B = B

    def forward(self, g, features, beta):
        global gcn_output
        n = nn.Softmax(dim=1)
        
        gcn_output = self.layer(g, features)
        weight_feature = n(gcn_output)
        W_noise = torch.div(torch.exp(weight_feature)-1, torch.exp(weight_feature)+1)*beta
        return W_noise
    
class GCN_prior(nn.Module):
    def __init__(self, A_hat, num_node, num_feat, num_cluster):
        super(GCN_prior, self).__init__()
        self.num_node = num_node # 特征数 f
        self.num_feat = num_feat
        self.num_cluster = num_cluster # 隐含数 h
        self.A_hat = A_hat
        self.d = torch.tensor(list(dict(G.degree()).values()), dtype=np.float).contiguous().view(-1, 1)
        self.ddT = torch.mm(self.d, self.d.T)
        self.B = (self.A_hat - torch.div(self.ddT, torch.norm(self.A_hat, p=1))).float()
        self.B_norm = torch.div(self.B, torch.norm(self.A_hat, p=1))
        self.W_0 = nn.Parameter(torch.ones(num_node, num_cluster)) # [1][f*h]
        self.layer = GCNLayer(num_feat, num_cluster)

    def forward(self, X, DG, temp, epoch): # X貌似暂时没有用到
        global featureSelector # 
        global weight_feature # 
        # f = nn.Softmax(dim=1)
        gcn_output = self.layer(DG, X)
        featureSelector = self.W_0 + gcn_output # [1][f*h]
        results = torch.zeros(featureSelector.size()) # [0][f*h]
        x = 1*(epoch+1) # x次取平均,目测这个目的是为了稳定。
        for i in range(x):
            # logits --> self.W_0: [batch_size, num_features] 非规范化对数概率
            # tau: 非负的对抗强度。
            # hard: 如果 True, 返回的样本将会离散为 one-hot 向量
            results += F.gumbel_softmax(featureSelector,tau=temp,hard=False)
        weight_feature = results/x # [-][f*h]

        # H = torch.mm(torch.mm(weight_feature.T, B), weight_feature) # weight_feature 就是我们要求的U
        H = torch.mm(torch.mm(weight_feature.T, self.A_hat), weight_feature) # weight_feature 就是我们要求的U
        Hh = torch.div(H, H.sum(axis=0)) # 按照每一列进行归一化，列为axis=0的方向。
        m = nn.Softmax(dim=0)
        return m(Hh)

In [81]:
W_prior_list = []

for G in multi_graph:

    A = torch.from_numpy(np.array(nx.adj_matrix(G).todense())).clone().float()
    features = torch.from_numpy(feature).clone().float()
    num_node = len(G.nodes()) # f = 节点数
    num_feat = features.size()[1]
    num_cluster = clusters_number # h = 聚类数
    model_base = GCN_prior(A, num_node, num_feat, num_cluster)
    optimizer_base = optim.Adam(model_base.parameters(),lr=1.5e-2)
    temp = 3
    epoch_prior_num = 200

    for epoch in range(epoch_prior_num):
        model_base.train()
        model_base.zero_grad()
        if(epoch == 75):
            temp = 2.75
        elif(epoch == 100):
            temp = 2.5
        elif(epoch == 125):
            temp = 2
        elif(epoch == 150):
            temp = 1.8
        elif(epoch == 175):
            temp = 1.25
        elif(epoch == 250):
            temp = 1.00
        elif(epoch == 300):
            temp = 0.75
        elif(epoch == 320):
            temp = 0.50
        elif(epoch == 400):
            temp = 0.20
        output = model_base(features, DG, temp, epoch)

        loss = lossFn(output)
        loss.backward(retain_graph=True)
        optimizer_base.step()

    gumbel_matrix = weight_feature.detach().max(dim=1)[1] # 每个节点的标签组成的tensor
    labels_pred = gumbel_matrix.data.numpy()
    # modu = community.modularity(dict(zip(list(range(len(labels_pred))), labels_pred)), G, weight='weight')
    # print('Epoch: ', epoch, '; Loss: ', round(loss.item(), 4), '; Modulariy: ', round(modu, 4))
    
    # 填装 prior
    partition = dict(zip(list(range(len(labels_pred))), labels_pred))
    partition_count = {}
    for key, value in partition.items():
        if value not in partition_count:
            partition_count[value] = 1
        else:
            partition_count[value] = partition_count[value] + 1

    # print(community.modularity(partition, G, weight='weight'))
    remapping = {}
    idx = 0
    for key, value in partition_count.items():
        if value>1 and key not in remapping:
            remapping[key] = idx
            idx = idx + 1

    tmp = np.zeros((len(G), clusters_number))
    for key, value in partition.items():
        try:
            tmp[key, remapping[value]] = alpha
        except:
            pass

    W_prior_list.append(tmp)

# 组合 prior
for i in range(len(W_prior_list)):
    W_prior_list[i] = torch.tensor(W_prior_list[i]).float().unsqueeze(0)
W_prior = torch.cat(tuple(W_prior_list), 0).to(device)

Epoch:  0 ; Loss:  11.8325 ; Modulariy:  0.6236
Epoch:  20 ; Loss:  11.8329 ; Modulariy:  0.6252
Epoch:  40 ; Loss:  11.8295 ; Modulariy:  0.6252
Epoch:  60 ; Loss:  11.829 ; Modulariy:  0.6252
Epoch:  80 ; Loss:  12.2256 ; Modulariy:  0.6206
Epoch:  100 ; Loss:  12.0092 ; Modulariy:  0.6252
Epoch:  120 ; Loss:  11.9589 ; Modulariy:  0.6252
Epoch:  140 ; Loss:  11.8628 ; Modulariy:  0.6252
Epoch:  160 ; Loss:  11.8446 ; Modulariy:  0.6252
Epoch:  180 ; Loss:  11.8297 ; Modulariy:  0.6252
Epoch:  200 ; Loss:  11.8297 ; Modulariy:  0.6252
Epoch:  220 ; Loss:  11.8296 ; Modulariy:  0.6252
Epoch:  240 ; Loss:  11.8296 ; Modulariy:  0.6252
Epoch:  260 ; Loss:  11.8284 ; Modulariy:  0.6252
Epoch:  280 ; Loss:  11.8285 ; Modulariy:  0.6252
Epoch:  300 ; Loss:  11.8281 ; Modulariy:  0.6252
Epoch:  320 ; Loss:  11.828 ; Modulariy:  0.6252
Epoch:  340 ; Loss:  11.828 ; Modulariy:  0.6252
Epoch:  360 ; Loss:  11.8281 ; Modulariy:  0.6252
Epoch:  380 ; Loss:  11.8281 ; Modulariy:  0.6252
Epoch:  4

In [ ]:
weight_list = torch.tensor(weight_list).float()
weight_list = torch.div(weight_list, torch.sum(weight_list))
weight_list = weight_list.to(device)

num_feat = len(multi_graph[0].nodes()) # f = 节点数
num_hidden = clusters_number # h = 聚类数

A_list = []
for i in multi_graph:
    A_list.append(torch.tensor(nx.adjacency_matrix(i).todense(), dtype=np.float).unsqueeze(0))
A_hat = A_hat = torch.cat(tuple(A_list), 0) # a matrix: A
A_hat_tensor = torch.Tensor(A_hat.float()).to(device)

vector_beta = torch.ones((feature.shape[1], 1)).float().to(device)
vector_omega = 1.0/clusters_number*torch.ones((clusters_number, 1)).float().to(device)
feature = torch.tensor(feature).float().to(device)

model = CDRAE(A_hat_tensor, num_feat, num_hidden, W_prior, weight_list).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-3)

temp = 3
for epoch in range(epoch_num):
    model.train()
    model.zero_grad()
    if(epoch == 75):
        temp = 2.75
    elif(epoch == 100):
        temp = 2.5
    elif(epoch == 125):
        temp = 2
    elif(epoch == 150):
        temp = 1.8
    elif(epoch == 175):
        temp = 1.25
    elif(epoch == 250):
        temp = 1.00
    elif(epoch == 300):
        temp = 0.75
    elif(epoch == 320):
        temp = 0.50
    elif(epoch == 400):
        temp = 0.20
    output = model(A_hat_tensor, temp, epoch)
    equality_o = torch.mm(torch.mm(weight_feature.T, feature),vector_beta)
    equality = equality_o/equality_o.sum()
    if epoch%10 == 0:
        P = torch.div(torch.square(weight_feature), torch.square(weight_feature).sum(axis=1).view(-1, 1))
    loss_clus = th_kl_div(weight_feature, P)
    # loss_pena = torch.sum(torch.mean(torch.square(featureSelector - torch.mean(featureSelector, axis=0).unsqueeze(0)), axis=0))
    loss_cons = torch.sqrt(torch.mean(torch.abs(equality - vector_omega))) # th_kl_div(equality, vector_omega)
    loss = alpha_1*lossFn(output) + alpha_2*loss_cons + alpha_3*loss_clus

    gumbel_matrix = weight_feature.clone().detach().max(dim=1)[1] # 每个节点的标签组成的tensor
    labels_pred = gumbel_matrix.cpu().data.numpy()
    ddd = dict(zip(list(range(len(labels_pred))), labels_pred))

    loss.backward(retain_graph=True)
    optimizer.step()

######################################################################################################
gumbel_matrix = weight_feature.clone().detach().max(dim=1)[1] # 每个节点的标签组成的tensor
labels_pred = gumbel_matrix.cpu().data.numpy()
partition = dict(zip(list(range(len(labels_pred))), list(labels_pred)))

return partition